## Programming Assignment: Размещение баннеров

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsq.zip с этой страницы.

Нас будет интересовать файл checkins.dat. 

In [1]:
import csv

with open('checkins.dat') as dat_file, open('checkins.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)

    for line in dat_file:
        row = [field.strip() for field in line.split('|')]
        if len(row) == 6 and row[3] and row[4]:
            csv_writer.writerow(row)

С помощью pandas построим DataFrame и убедимся, что все 396634 строки с координатами считаны успешно.

In [2]:
import pandas as pd
import numpy as np
import matplotlib

data = pd.read_csv('checkins.csv')
data_100k = data.loc[:99999]
print(data.shape)
print(data.columns)
print(data_100k.shape)

(396634, 6)
Index(['id', 'user_id', 'venue_id', 'latitude', 'longitude', 'created_at'], dtype='object')
(100000, 6)


Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [3]:
from sklearn.cluster import MeanShift

coors_100k_df = data_100k[['latitude', 'longitude']]
coors_100k_list = coors_100k_df.values.tolist()

clustering = MeanShift(bandwidth=0.1).fit(coors_100k_list)

MeanShift(bandwidth=0.1, bin_seeding=False, cluster_all=True, min_bin_freq=1,
          n_jobs=None, seeds=None)


In [4]:
data_100k['cluster'] = clustering.labels_
data_100k.head()

/Users/bs/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,user_id,venue_id,latitude,longitude,created_at,cluster
0,984222,15824,5222,38.895112,-77.036366,2012-04-21 17:43:47,5
1,984234,44652,5222,33.800745,-84.410520,2012-04-21 17:43:43,7
2,984291,105054,5222,45.523452,-122.676207,2012-04-21 17:39:22,30
3,984318,2146539,5222,40.764462,-111.904565,2012-04-21 17:35:46,65
4,984232,93870,380645,33.448377,-112.074037,2012-04-21 17:38:18,1


Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

In [5]:
from haversine import haversine, Unit

offices_coors = [(33.751277, -118.188740), (25.867736, -80.324116), (51.503016, -0.075479),(52.378894, 4.885084),(39.366487, 117.036146),(-33.868457, 151.205134)]

data_grouped_filtered_clusters = data_100k.groupby('cluster').size()

clusters = pd.DataFrame({'cluster':data_grouped_filtered_clusters.index, 'size':data_grouped_filtered_clusters.values})
clusters['center'] = clusters.apply(lambda row: clustering.cluster_centers_[row['cluster']], axis = 1) 

filtered_clusters = clusters[clusters['size'] > 15]
filtered_clusters['min_dst_to office'] = clusters.apply(lambda row: min([haversine(row['center'], office) for office in offices_coors]), axis = 1) 



/Users/bs/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [17]:
sorted_filtered_clusters = filtered_clusters.sort_values(by=['min_dst_to office'])
nearest_cluster_center = sorted_filtered_clusters.iloc[0]['center']

print(nearest_cluster_center)

with open("ans.txt", "w") as fout:
    fout.write(' '.join([str(x) for x in nearest_cluster_center]))

[52.37296399  4.89231722]
